In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("stockprice_15_04_2019-25_10_2017.csv")
data.columns = ["time", "traded_companies", "no_trans", "high", 
                "low", "close", "traded_shares", "amount", "open", "diff"]
                #"ma20", "ma50", "std20", "bband_l", "bband_u", "mad"]
#data.set_index("time", inplace = True)


In [5]:
from collections import Counter
Counter((data['time'] + data['traded_companies']).to_numpy()).most_common()


[('4/15/2010Api Finance Limited', 1),
 ('4/15/2010Arun Valley Hydropower Development Co. Ltd.', 1),
 ('4/15/2010Axis Development Bank Limited', 1),
 ('4/15/2010Bank of Asia Nepal Limited', 1),
 ('4/15/2010Bank of Kathmandu Ltd.', 1),
 ('4/15/2010Bhrikuti Vikash Bank Limited', 1),
 ('4/15/2010Bishal Bazar Company Limited', 1),
 ('4/15/2010Business Universal Development Bank Ltd.', 1),
 ('4/15/2010Butwal Power Company Limited', 1),
 ('4/15/2010Chilime Hydropower Company Limited', 1),
 ('4/15/2010Citizen Bank International Limited', 1),
 ('4/15/2010Crystal Finance Ltd.', 1),
 ('4/15/2010Deva Bikas Bank Limited', 1),
 ('4/15/2010Everest Bank Limited', 1),
 ('4/15/2010Everest Insurance Co. Ltd.', 1),
 ('4/15/2010Excel Development Bank Ltd.', 1),
 ('4/15/2010Fewa Bikas Bank Ltd. Promoter Share', 1),
 ('4/15/2010Global IME Bank Limited', 1),
 ('4/15/2010Gorkha Finance Ltd.', 1),
 ('4/15/2010Grand Bank Nepal Limited', 1),
 ('4/15/2010Gurkha Development Bank Limited', 1),
 ('4/15/2010Gurkhas Fi

In [4]:
data = data[data.traded_companies != 'Rastriya Beema Company Limited']
data = data[data.traded_companies != 'Capital Merchant Bank  Finance Co. Ltd.']
data = data[data.traded_companies != 'Kumari Bank Limited Promotor Share']
data.dropna(inplace = True)
data.tail()

,time,traded_companies,no_trans,high,low,close,traded_shares,amount,open,diff
182492,10/25/2017,United Insurance Co. (Nepal) Ltd.,18.0,1295.0,1280.0,1284.0,1685.0,2168340.0,1284.0,0.0
182493,10/25/2017,United Modi Hydropower Ltd.,9.0,258.0,253.0,256.0,601.0,154326.0,256.0,0.0
182494,10/25/2017,Vijaya laghubitta Bittiya Sanstha Ltd.,8.0,1248.0,1230.0,1248.0,283.0,351253.0,1241.0,7.0
182495,10/25/2017,Womi Microfinance Bittiya Sanstha Ltd.,9.0,2010.0,1980.0,1980.0,396.0,786340.0,2020.0,-40.0
182496,10/25/2017,World Merchant Banking Finance Ltd.,8.0,136.0,134.0,134.0,1560.0,210220.0,138.0,-4.0


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180990 entries, 0 to 180989
Data columns (total 19 columns):
time                180990 non-null float64
traded_companies    180990 non-null object
no_trans            180990 non-null float64
high                180990 non-null float64
low                 180990 non-null float64
close               180990 non-null float64
traded_shares       180990 non-null float64
amount              180990 non-null float64
open                180990 non-null float64
diff                180990 non-null float64
rsi14               175822 non-null float64
ma9                 177867 non-null float64
ma20                174266 non-null float64
ma50                165614 non-null float64
std20               174266 non-null float64
bband_l             174266 non-null float64
bband_u             174266 non-null float64
color               180990 non-null object
macd920             174266 non-null float64
dtypes: float64(17), object(2)
memory usage: 27.6+ MB


In [7]:
def RSI(stock_cp, window=14):
    # Calculating upward and downward trends
    stock_cp = stock_cp
    up_trends = stock_cp.rolling(window=2).apply(lambda y: y[1] - y[0] if y[1] > y[0] else 0)
    down_trends = stock_cp.rolling(window=2).apply(lambda y: y[0] - y[1] if y[0] > y[1] else 0)

    # Averaging upward and downward trends over period of window
    avg_uptrends = up_trends.rolling(window=window).mean()
    # print(avg_uptrends.head(30))
    avg_downtrends = down_trends.rolling(window=window).mean()
    # print(avg_downtrends.head(30))

    # Finally RSI
    RSI = 100 - 100/(1+avg_uptrends.divide(avg_downtrends))
    return RSI


In [8]:
rsi14 = data[["time","traded_companies", "close"]].set_index("time").groupby(["traded_companies"]).pipe(RSI).reset_index()
rsi14.columns = ["traded_companies","time", "rsi14"]
data = pd.merge(data,rsi14, on=["time", "traded_companies"],how='inner')

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  after removing the cwd from sys.path.
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """


In [9]:
ma9 = data[["time","traded_companies", "close"]].set_index("time").groupby(["traded_companies"]).rolling(9).mean().reset_index()
ma9.columns = ["traded_companies","time", "ma9"]
data = pd.merge(data,ma9, on=["time", "traded_companies"],how='inner')

In [10]:
ma20 = data[["time","traded_companies", "close"]].set_index("time").groupby(["traded_companies"]).rolling(20).mean().reset_index()
ma20.columns = ["traded_companies","time", "ma20"]
data = pd.merge(data,ma20, on=["time", "traded_companies"],how='inner')

In [14]:
data.tail(100)

,time,traded_companies,no_trans,high,low,close,traded_shares,amount,open,diff,rsi14,ma9,ma20,ma50,std20
180890,10/25/2017,Kumari Bank Limited,128.0,315.00,309.00,312.00,43801.0,13672242.0,310.00,2.00,58.208955,307.555556,301.2500,305.5400,8.515466
180891,10/25/2017,Laxmi Bank Limited,31.0,333.00,322.00,326.00,3234.0,1056888.0,330.00,-4.00,39.393939,326.444444,328.5000,338.8800,7.796760
180892,10/25/2017,Laxmi Laghubitta Bittiya Sanstha Ltd.,10.0,1650.00,1620.00,1645.00,751.0,1229520.0,1680.00,-35.00,46.736292,1624.555556,1641.3500,1885.1400,48.879309
180893,10/25/2017,Laxmi Value Fund-1,7.0,11.80,11.51,11.60,52000.0,605765.0,11.70,-0.10,60.606061,11.581111,11.3970,12.0542,0.232856
180894,10/25/2017,Life Insurance Co. Nepal,24.0,2125.00,2095.00,2095.00,2154.0,4525141.0,2134.00,-39.00,51.012146,2096.777778,2083.9000,2192.8400,26.550647
180895,10/25/2017,Lumbini Bikas Bank Ltd.,31.0,214.00,211.00,212.00,6070.0,1291068.0,214.00,-2.00,48.571429,214.000000,211.2500,213.2200,3.891624
180896,10/25/2017,Lumbini General Insurance Co. Ltd.,77.0,870.00,859.00,860.00,9625.0,8285966.0,865.00,-5.00,30.964467,873.000000,894.6500,950.8400,27.434659
180897,10/25/2017,Machhapuchhre Bank Limited,50.0,326.00,319.00,319.00,11312.0,3662195.0,324.00,-5.00,43.478261,321.888889,321.8500,329.8200,4.880412
180898,10/25/2017,Mahalaxmi Bikas Bank Ltd.,23.0,225.00,221.00,223.00,2109.0,468427.0,224.00,-1.00,45.161290,223.888889,223.0500,230.0400,3.252125
180899,10/25/2017,Manjushree Finance Ltd.,11.0,150.00,145.00,149.00,2551.0,374140.0,148.00,1.00,55.882353,148.000000,147.3000,151.6800,2.273416


In [12]:
ma50 = data[["time","traded_companies", "close"]].set_index("time").groupby(["traded_companies"]).rolling(50).mean().reset_index()
ma50.columns = ["traded_companies","time", "ma50"]
data = pd.merge(data,ma50, on=["time", "traded_companies"],how='inner')

In [13]:
std20 = data[["time","traded_companies", "close"]].set_index("time").groupby(["traded_companies"]).rolling(20).std().reset_index()
std20.columns = ["traded_companies","time", "std20"]
data = pd.merge(data,std20, on=["time", "traded_companies"],how='inner')

In [15]:

data.loc[data['traded_companies'] == 'Nabil Bank Limited'].head(50)

,time,traded_companies,no_trans,high,low,close,traded_shares,amount,open,diff,rsi14,ma9,ma20,ma50,std20
34,4/15/2010,Nabil Bank Limited,40.0,2040.0,2000.0,2000.0,3464.0,6935470.0,2040.0,-40.0,NaN,NaN,NaN,NaN,NaN
87,4/19/2010,Nabil Bank Limited,18.0,2039.0,2000.0,2001.0,1774.0,3549542.0,2080.0,-79.0,NaN,NaN,NaN,NaN,NaN
145,4/20/2010,Nabil Bank Limited,18.0,1975.0,1945.0,1945.0,1212.0,2369304.0,2001.0,-56.0,NaN,NaN,NaN,NaN,NaN
199,4/21/2010,Nabil Bank Limited,35.0,1908.0,1800.0,1810.0,1916.0,3531917.0,1945.0,-135.0,NaN,NaN,NaN,NaN,NaN
256,4/22/2010,Nabil Bank Limited,17.0,1831.0,1775.0,1831.0,1197.0,2156607.0,1810.0,21.0,NaN,NaN,NaN,NaN,NaN
313,4/25/2010,Nabil Bank Limited,16.0,1804.0,1770.0,1786.0,1365.0,2440702.0,1831.0,-45.0,NaN,NaN,NaN,NaN,NaN
369,4/26/2010,Nabil Bank Limited,14.0,1785.0,1750.0,1760.0,920.0,1625754.0,1786.0,-26.0,NaN,NaN,NaN,NaN,NaN
432,4/27/2010,Nabil Bank Limited,17.0,1730.0,1695.0,1696.0,1418.0,2413719.0,1760.0,-64.0,NaN,NaN,NaN,NaN,NaN
488,4/28/2010,Nabil Bank Limited,5.0,1729.0,1665.0,1671.0,293.0,491761.0,1696.0,-25.0,NaN,1833.333333,NaN,NaN,NaN
550,4/29/2010,Nabil Bank Limited,12.0,1838.0,1704.0,1838.0,622.0,1132280.0,1671.0,167.0,NaN,1815.333333,NaN,NaN,NaN


In [16]:
data["bband_l"] = data["ma20"] - 2* data["std20"]
data["bband_u"] = data["ma20"] + 2* data["std20"]

In [17]:
data["color"] = np.where(data['diff'] > 0, "green", "red")

In [18]:
def ema(df, span = 9):
    return df.ewm(span=span).mean()

In [19]:
#ema9 = data[["time","traded_companies", "close"]].set_index("time").groupby(["traded_companies"]).apply(ema).reset_index()
#ema9

In [20]:
data["macd920"] = data["ma9"] - data["ma20"]

In [21]:
data["time"] = np.array(pd.to_datetime(data['time'], cache = True),dtype = np.datetime64).tolist()
data["time"] = data["time"].apply(lambda x: x/1000000)

In [23]:
data.head()

,time,traded_companies,no_trans,high,low,close,traded_shares,amount,open,diff,rsi14,ma9,ma20,ma50,std20,bband_l,bband_u,color,macd920
0,1.271290e+12,Api Finance Limited,46.0,247.0,240.0,240.0,1520.0,375070.0,243.0,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,red,NaN
1,1.271290e+12,Arun Valley Hydropower Development Co. Ltd.,2.0,430.0,430.0,430.0,30.0,12900.0,438.0,-8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,red,NaN
2,1.271290e+12,Axis Development Bank Limited,2.0,240.0,240.0,240.0,60.0,14400.0,240.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,red,NaN
3,1.271290e+12,Bank of Asia Nepal Limited,7.0,311.0,300.0,300.0,440.0,134040.0,308.0,-8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,red,NaN
4,1.271290e+12,Bank of Kathmandu Ltd.,6.0,747.0,740.0,747.0,273.0,202424.0,740.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,green,NaN


In [24]:
data.dropna().to_csv("processed_data.csv",index=False)

In [117]:
from bokeh.models import ColumnDataSource

In [118]:
source = ColumnDataSource(data)

In [121]:
str(source)

"ColumnDataSource(id='1001', ...)"